<a href="https://colab.research.google.com/github/raima2000/Pharmacovigilance_detection/blob/main/Biobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:

df=pd.read_excel("/content/psyTAR_selected.xlsx")
df.head()

,comment,ADR_Status
0,I am detoxing from Lexapro now. I slowly cut m...,ADR
1,Just TWO tablets of Lexapro 10mg completely de...,ADR
2,Be careful with this medication. This was my ...,ADR
3,"It didn't help me out, at all. My anxiety is w...",ADR
4,I would not suggest taking this medication. I ...,ADR


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Assuming df1 is your DataFrame with 'comment' and 'ADR_Status' columns

# Encode labels
label_encoder = LabelEncoder()
df['ADR_Status_encoded'] = label_encoder.fit_transform(df['ADR_Status'])

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 12.5 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFBertForSequenceClassification

# Load BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
model = TFBertForSequenceClassification.from_pretrained("dmis-lab/biobert-v1.1", num_labels=2,from_pt=True)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# import pandas as pd
# import tensorflow as tf
# from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# # Load BERT tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2, from_pt=True)



# Tokenize and prepare data
max_length = 128

def tokenize_text(text):
    inputs = tokenizer(
        text,
        add_special_tokens=True,
        truncation=True,
        max_length=max_length,
        padding='max_length',
        return_tensors='tf'
    )
    return inputs

X = df['comment'].tolist()
y = df['ADR_Status_encoded'].tolist()  # Ensure labels are in integer format

# Split the data into training and testing sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text data for both training and testing
X_train_encoded = [tokenize_text(text) for text in X_train]
X_test_encoded = [tokenize_text(text) for text in X_test]

# Extract input_ids from X_encoded
X_train_input_ids = [input_dict['input_ids'][0] for input_dict in X_train_encoded]
X_test_input_ids = [input_dict['input_ids'][0] for input_dict in X_test_encoded]

# Pad the sequences to have consistent lengths
X_train_input_ids = tf.keras.preprocessing.sequence.pad_sequences(X_train_input_ids, maxlen=max_length, padding='post')
X_test_input_ids = tf.keras.preprocessing.sequence.pad_sequences(X_test_input_ids, maxlen=max_length, padding='post')

y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

# Define training arguments
train_args = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Create a model
model.compile(optimizer=train_args, loss=loss_fn, metrics=['accuracy'])

# Train the model
model.fit(
    X_train_input_ids,
    y_train,
    epochs=5,
    batch_size=32,
    validation_split=0.2
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_input_ids, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Save the model
model.save_pretrained('fine_tuned_model')
tokenizer.save_pretrained('fine_tuned_model')




Epoch 1/5
16/16 [==============================] - 1070s 61s/step - loss: 0.4274 - accuracy: 0.8147 - val_loss: 0.1998 - val_accuracy: 0.9512
Epoch 2/5
16/16 [==============================] - 980s 62s/step - loss: 0.2164 - accuracy: 0.9104 - val_loss: 0.2097 - val_accuracy: 0.9106
Epoch 3/5
16/16 [==============================] - 1002s 63s/step - loss: 0.2045 - accuracy: 0.9084 - val_loss: 0.1647 - val_accuracy: 0.9512
Epoch 4/5
16/16 [==============================] - 956s 60s/step - loss: 0.1652 - accuracy: 0.9226 - val_loss: 0.1649 - val_accuracy: 0.9593
Epoch 5/5
5/5 [==============================] - 83s 16s/step - loss: 0.1708 - accuracy: 0.9416
Test Loss: 0.1708
Test Accuracy: 0.9416


('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/vocab.txt',
 'fine_tuned_model/added_tokens.json',
 'fine_tuned_model/tokenizer.json')

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Make predictions on the test set
y_pred = model.predict(X_test_input_ids)
y_pred_classes = np.argmax(y_pred.logits, axis=1)

# Calculate precision, recall, and F1 score
precision = precision_score(y_test, y_pred_classes)
recall = recall_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


5/5 [==============================] - 97s 17s/step
Precision: 0.75
Recall: 0.2727272727272727
F1 Score: 0.39999999999999997


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix


## Calculate precision, recall, F1-score, and support
report = classification_report(y_test, y_pred_classes, target_names=['Class 0', 'Class 1'])
print("Classification Report:\n", report)



Classification Report:
               precision    recall  f1-score   support

     Class 0       0.95      0.99      0.97       143
     Class 1       0.75      0.27      0.40        11

    accuracy                           0.94       154
   macro avg       0.85      0.63      0.68       154
weighted avg       0.93      0.94      0.93       154



In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# # Calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

Confusion Matrix:
 [[142   1]
 [  8   3]]
